# Introduction to the Azure ML SDK

Azure Machine Learning (*Azure ML*) is a cloud-based service for creating and managing machine learning solutions. It's designed to help data scientists leverage their existing data processing and model development skills and frameworks, and help them scale their workloads to the cloud. The Azure ML SDK for Python provides classes you can use to work with Azure ML in your Azure subscription.

## Check the Azure ML SDK Version

Let's start by importing the **azureml-core** package and checking the version of the SDK that is installed.

Run the `pip3` command if the `import` fails.  Also, make sure if you have multiple kernels installed in your env that your pip3 command is referencing the running kernel.  

In [ ]:
# !pip3 install azureml-sdk

In [1]:
import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

Ready to use Azure ML 1.18.0


In [2]:
import sys
print('You are using Python', sys.version)

You are using Python 3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) 
[GCC 7.3.0]


Sometimes we release new versions of the SDK and you may not be running those in your AzureML compute environment if you haven't build a new personal compute instance in a while.  Always best to check and rebuild the compute every few weeks.  **Rebuilding the compute will not cause you to lose your data and storage**.  

In [3]:
pip list | grep azureml

azureml-accel-models                  1.18.0
azureml-automl-core                   1.18.0
azureml-automl-runtime                1.18.0
azureml-cli-common                    1.18.0
azureml-contrib-dataset               1.18.0
azureml-contrib-fairness              1.18.0
azureml-contrib-gbdt                  1.18.0
azureml-contrib-interpret             1.18.0
azureml-contrib-notebook              1.18.0
azureml-contrib-pipeline-steps        1.18.0
azureml-contrib-reinforcementlearning 1.18.0
azureml-contrib-server                1.18.0
azureml-contrib-services              1.18.0
azureml-core                          1.18.0
azureml-datadrift                     1.18.0
azureml-dataprep                      2.4.2
azureml-dataprep-native               24.0.0
azureml-dataprep-rslex                1.2.2
azureml-dataset-runtime               1.18.0
azureml-defaults                      1.18.0
azureml-explain-model                 1.18.0
azureml-interpret                     1.18.0
azureml-mlfl

In [4]:
import azureml.core 
print ("You are using Azure ML", azureml.core.VERSION)

You are using Azure ML 1.18.0


If you need to upgrade Azure ML...

In [5]:
#!pip install --upgrade azureml-sdk
#!pip install --upgrade azureml-sdk[automl,notebooks,explain,contrib,services,tensorboard]

## Connect to Your Workspace

All experiments and associated resources are managed within you Azure ML workspace. You can connect to an existing workspace, or create a new one using the Azure ML SDK.

In most cases, you should store the workspace configuration in a JSON configuration file. This makes it easier to reconnect without needing to remember details like your Azure subscription ID. You can download the JSON configuration file from the blade for your workspace in the Azure portal, but if you're using a Compute Instance within your wokspace, the configuration file has already been downloaded to the root folder.

The code below uses the configuration file to connect to your workspace. The first time you run it in a notebook session, you'll be prompted to sign into Azure by clicking the https://microsoft.com/devicelogin link,  entering an automatically generated code, and signing into Azure. After you have successfully signed in, you can close the browser tab that was opened and return to this notebook.

If you don't have a workspace config file, here's how you get it:

* open your AMLS instance in the azure portal, at the top you'll see `Download config.json`
* save that file to `.azureml` in the root of your project
* rerun the cell

**Note: I have my personal config.json in the root of this folder, so you can just replace that**

In [6]:
from azureml.core import Workspace

ws = Workspace.from_config()

print('Workspace configuration succeeded')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code R7FQAVZZR to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
WARNING - Failed to authenticate to tenant '354e6dce-302d-4d66-9ae3-8947df67c030' due to error 'Get Token request returned http error: 400 and server response: {"error":"interaction_required","error_description":"AADSTS50076: Due to a configuration change made by your administrator, or because you moved to a new location, you must use multi-factor authentication to access '797f4846-ba00-4fd7-ba43-dac1f8f63013'.\r\nTrace ID: ae4f9310-019b-4e2f-9be2-e874e2581700\r\nCorrelation ID: e2f9dc16-b5b9-476d-ba1c-6f2550987340\r\nTimestamp: 2021-03-03 16:40:15Z","error_codes":[50076],"timestamp":"2021-03-03 16:40:15Z","trace_id":"ae4f9310-019b-4e2f-9be2-e874e2581700","correlation_id":"e2f9dc16-b5b9-476d

Here is an alternative way, in code, to write out your config file

In [ ]:
import os
subscription_id = os.environ.get("SUBSCRIPTION_ID", "<changeme>")
resource_group = os.environ.get("RESOURCE_GROUP", "<changeme>")
workspace_name = os.environ.get("WORKSPACE_NAME", "<changeme>")

from azureml.core import Workspace
try:
   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
   ws.write_config()
   print("OK")
except:
   print("Error: Workspace not found")

In [8]:
# get workspace information if you aren't sure which workspace you are connected to
ws = Workspace.from_config()
ws.get_details()

{'id': '/subscriptions/52061d21-01dd-4f9e-aca9-60fff4d67ee2/resourceGroups/MLOpsWorkshop/providers/Microsoft.MachineLearningServices/workspaces/mlops',
 'name': 'mlops',
 'location': 'eastus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': 'ff8ef3fc-dbd9-4bd7-8d8a-d734bf3dfc40',
 'sdkTelemetryAppInsightsKey': 'f5784ccd-178d-4ecc-9998-b05841b44ae9',
 'description': '',
 'friendlyName': '',
 'creationTime': '2019-10-23T15:45:18.1770440+00:00',
 'containerRegistry': '/subscriptions/52061d21-01dd-4f9e-aca9-60fff4d67ee2/resourceGroups/MLOpsWorkshop/providers/Microsoft.ContainerRegistry/registries/mlopse1388ade',
 'keyVault': '/subscriptions/52061d21-01dd-4f9e-aca9-60fff4d67ee2/resourcegroups/mlopsworkshop/providers/microsoft.keyvault/vaults/mlops1134511691',
 'applicationInsights': '/subscriptions/52061d21-01dd-4f9e-aca9-60fff4d67ee2/resourcegroups/mlopsworkshop/providers/microsoft.insights/components/mlops1128078900',
 'identityPrinci

In [7]:
## Session Information
## this is also valuable if you think you may have a library mismatch
import scipy, numpy, matplotlib, pandas, sklearn
print("Versions :")
print('- Python = {}'.format(sys.version))
print('- scipy = {}'.format(scipy.__version__))
print('- numpy = {}'.format(numpy.__version__))
print('- matplotlib = {}'.format(matplotlib.__version__))
print('- pandas = {}'.format(pandas.__version__))
print('- sklearn = {}'.format(sklearn.__version__))
print("- Azure ML =",azureml.core.VERSION)

Versions :
- Python = 3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) 
[GCC 7.3.0]
- scipy = 1.5.2
- numpy = 1.18.5
- matplotlib = 3.2.1
- pandas = 1.1.4
- sklearn = 0.23.2
- Azure ML = 1.18.0


## View Azure ML Resources

Now that you have a connection to your workspace, you can view the resources it contains.

In [9]:
from azureml.core import ComputeTarget, Datastore, Dataset

print("Compute Targets:")
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print("\t", compute.name, ':', compute.type)
    
print("Datastores:")
for datastore_name in ws.datastores:
    datastore = Datastore.get(ws, datastore_name)
    print("\t", datastore.name, ':', datastore.datastore_type)
    
print("Datasets:")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name)

Compute Targets:
	 infer-cluster : AKS
	 gpucluster : AmlCompute
	 automl : AmlCompute
	 automl-big : AmlCompute
	 davew-compute : ComputeInstance
	 davew-small : ComputeInstance
	 dturner-compute : ComputeInstance
	 davew : ComputeInstance
	 rross : ComputeInstance
	 davew2 : ComputeInstance
	 big : ComputeInstance
	 mem-big : ComputeInstance
Datastores:
	 davewdemoblobs : AzureBlob
	 azureml_globaldatasets : AzureBlob
	 workspacefilestore : AzureFile
	 workspaceblobstore : AzureBlob
Datasets:
	 customer488
	 synovos-curated488
	 synovos488class-tabular
	 synovos-curated
	 synovos-cat-items
	 accts_receivable
	 ar-factoring-2class
	 ar-factoring
	 data_train_experiment
	 diabetes target
	 diabetes baseline
	 diabetes dataset
	 avantor-10prods
	 ts-avantor
	 merged
	 attrition-dataset
	 connected_car_components
	 glove_6B_100d


Now you've seen how to use the Azure ML SDK to view the resources in your workspace. The SDK provides a great way to script the creation and configuration of the resources you need to operate machine learning workloads using Azure ML. For more details, see the [Azure ML SDK documentation](https://docs.microsoft.com/python/api/overview/azure/ml/intro?view=azure-ml-py).